In [1]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# pipeline
from sklearn.pipeline import Pipeline

# for the preprocessors
from sklearn.base import BaseEstimator, TransformerMixin

#for stats
import scipy.stats as stats

In [2]:
# load data
train_x = pd.read_csv('data/xtrain.csv')

train_y = pd.read_csv('data/ytrain.csv')

In [3]:
train_x

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,6.696685,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,48.629489,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,57.116252,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,56.083096,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,55.905122,1212,1411,8,2,15,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,14,56.216165,1890,668,13,4,19,1,1,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,3,48.888851,1965,2032,11,10,16,1,1,1
1997,1911,0,0.9,1,1,1,36,0.7,108,8,3,47.656262,1632,3057,9,1,5,1,1,0
1998,1512,0,0.9,0,4,1,46,0.1,145,5,5,29.938332,670,869,18,10,19,1,1,1


Config pipeline

In [12]:
BINARY = ['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']
DISCRETE_VARS = ['fc', 'n_cores', 'pc', 'ram', 'sc_h', 'sc_w', 'talk_time']
CONTINUE_VARS = ['clock_speed', 'battery_power', 'battery_power', 'm_dep', 'mobile_wt', 
                'px_width', 'ram']
YEOJOHNSON = ['px_height']

In [10]:
class YeojohnsonTransformer(BaseEstimator, TransformerMixin):
    # Extract fist letter of variable

    def __init__(self, variables):
        
        if not isinstance(variables, list):
            raise ValueError('variables should be a list')
        
        self.variables = variables
        


    def fit(self, X, y=None):
        # we need this step to fit the sklearn pipeline
        return self

        

    def transform(self, X):

        # so that we do not over-write the original dataframe
        X = X.copy()
        
        for feature in self.variables:
            X[feature], param = stats.yeojohnson(X[feature])

        return X

In [14]:
# set up the pipeline
phone_pipe = Pipeline([
    
    #Yeojohnson
    ('Yeojohnson', YeojohnsonTransformer(variables=YEOJOHNSON)),

    # scale
    ('scaler', StandardScaler()),

    ('Logit', LogisticRegression(C=0.0005, random_state=0)),
])

In [15]:
pd.pandas.set_option('display.max_columns', None)

In [16]:
phone_pipe.fit(train_x, train_y)

C:\Users\gonz3\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Pipeline(steps=[('Yeojohnson', YeojohnsonTransformer(variables=['px_height'])),
                ('scaler', StandardScaler()),
                ('Logit', LogisticRegression(C=0.0005, random_state=0))])